# TODO

### 위스콘신 유방암 데이터셋
- 위스콘신 대학교에서 제공한 유방암 진단결과 데이터
- Feature: 종양 측정값들
    - 모든 Feature들은 연속형(continous)이다.
- target: 악성, 양성 여부
- scikit-learn에서 toy dataset으로 제공한다. 
    - load_breast_cancer() 함수 이용<br><br>

> '위스콘신 유방암 데이터셋'을 이용해 Feature Scaling의 두가지 방법(Standard Scaling, MinMax Scaling)을 실행하고
각각의 변환 데이터셋으로 `SVC`모델링을 하고 `accuracy_score` 확인하기.<br><br>
    
- StandardScaler와 MinMax Scaler를 이용해 위스콘신 유방암 데이터셋의 Feature들 scaling 처리를 한다.
    - Scaler 학습은 Train set으로 만 하고 그 학습된 것을 이용해 Train/Validation/Test set을 변환한다.
- **StandardScaler 로 변환한 결과를 저장할 변수**
    - X_train_scaled1, X_val_scaled1, X_test_scaled1
- **MinMaxScaler 로 변환한 결과를 저장할 변수**
    - X_train_scaled2, X_val_scaled2, X_test_scaled2

In [1]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

# 데이터 로드
from sklearn.datasets import load_breast_cancer

data = load_breast_cancer()
X = data['data']
y = data['target']
X.shape, y.shape

((569, 30), (569,))

In [2]:
np.unique(y, return_counts=True)

(array([0, 1]), array([212, 357], dtype=int64))

In [3]:
# 0: 악성 , 1: 양성
data['target_names']

array(['malignant', 'benign'], dtype='<U9')

In [4]:
data['feature_names']

array(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error',
       'fractal dimension error', 'worst radius', 'worst texture',
       'worst perimeter', 'worst area', 'worst smoothness',
       'worst compactness', 'worst concavity', 'worst concave points',
       'worst symmetry', 'worst fractal dimension'], dtype='<U23')

In [5]:
# train validation test set 분리

X_tmp, X_test, y_tmp, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=0)

X_train, X_val, y_train, y_val = train_test_split(X_tmp, y_tmp, 
                                                  test_size=0.2, 
                                                  stratify=y_tmp, random_state=0)

X_train.shape, X_val.shape, X_test.shape

((364, 30), (91, 30), (114, 30))

In [10]:
# Standard Scaling
# 객체 생성
s_scaler = StandardScaler()

# 학습 및 변환
X_train_scaled1 = s_scaler.fit_transform(X_train)
X_val_scaled1 = s_scaler.transform(X_val)
X_test_scaled1 = s_scaler.transform(X_test)

# 각 feature 들의 mean 이 0에 가깝게, std 가 1에 가깝게 변환됨

In [12]:
# MinMax Scaling
# 객체 생성
mm_scaler = MinMaxScaler()

# 학습 및 변환
X_train_scaled2 = mm_scaler.fit_transform(X_train)
X_val_scaled2 = mm_scaler.transform(X_val)
X_test_scaled2 = mm_scaler.transform(X_test)

# X_train은 min(0) max(1) 값들로 변환되었고,
# val, test는 train의 min/max 기준으로 scaling이 진행되었기 때문에
# 초과되는 값은 1보다 크게, 미만인 값은 0보다 작게, 변환되었지만
# 그리 큰 차이를 보여주지 않아 해당 데이터셋은 MinMax Scaling으로도 모델링을 할만하다.

In [13]:
# SVC 모델링, accuracy 평가지표
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [14]:
# Scaling을 진행하지 않은 데이터셋 모델링, 테스트용
svm = SVC(random_state=0, C=0.1, gamma=0.1)
# 학습
svm.fit(X_train, y_train)
# 검증
## 추론
pred_train = svm.predict(X_train)
pred_val = svm.predict(X_val)
## 평가 - 정확도
print("train:", accuracy_score(y_train, pred_train))
print("validation:", accuracy_score(y_val, pred_val))

train: 0.6263736263736264
validation: 0.6263736263736264


In [20]:
# Standard Scaling을 한 데이터셋 모델링
svm = SVC(random_state=0, C=0.1, gamma=0.1)

# 학습
svm.fit(X_train_scaled1, y_train)

## 추론
pred_train1 = svm.predict(X_train_scaled1)
pred_val1 = svm.predict(X_val_scaled1)

## 평가 - 정확도
print("train", accuracy_score(y_train,pred_train1))
print("validation", accuracy_score(y_val, pred_val1))

train 0.9560439560439561
validation 0.9340659340659341


In [21]:
# MinMax Scaling 데이터셋 모델링

# 모델 생성
svm = SVC(random_state=0, C=0.1, gamma=0.1)

# 학습
svm.fit(X_train_scaled2, y_train)

# 검증
## 추론
pred_train2 = svm.predict(X_train_scaled2)
pred_val2 = svm.predict(X_val_scaled2)

## 평가 - 정확도
print("train:", accuracy_score(y_train, pred_train2))
print("validation:", accuracy_score(y_val, pred_val2))

train: 0.9175824175824175
validation: 0.9010989010989011


- 위의 결과값에 따라서 최종평가에 사용될 데이터셋은<br>가장 결과가 좋은 'Standard Scaling' 데이터셋을 사용할 것
---

# TODO adult dataset 모델링

- 전처리
    - 범주형: one hot encoding
    - 연속형: standard scaling
- 모델
    - sklearn.linear_model.**LogisticRegression(max_iter=1000, random_state=0)**
    - sklearn.svm.**SVC(random_state=0)**
- train/test dataset으로 나누고 train set으로 cross validation 학습 및 검증을 하고 test set으로 최종 평가 진행

**데이터 학습-모델링 순서** :
1. import
2. data loading
3. 결측치 처리
4. input, output data 분리
5. 범주형 feature 전처리
6. train test set 분리
7. 연속형 feature 전처리
8. 모델링

### 🚩 연속형 Feature 전처리 시, `Train, (Validation), Test set` 으로 분할한 후 진행한다.
> Why? -> Validation 과 Test set 들은 앞으로 예측하게 될 **새로운 데이터**에 대해 어느정도의 성능인지를 파악하기 위한 평가를 하는 데이터셋들이다. 그러나 이들이 **같은 scale을 가지는 지 보장할 수 없기 때문에**<br>
`전체 데이터셋을 Scaling`하고 Train, Validation, Test set 으로 나누는 것은 **모델의 성능 파악이 어렵다.**

In [22]:
cols = ['age', 'workclass','fnlwgt','education', 'education-num', 'marital-status', 'occupation','relationship', 'race', 'gender','capital-gain','capital-loss', 'hours-per-week','native-country', 'income']

# 원핫인코딩
category_columns = ['workclass','education','marital-status', 'occupation','relationship','race','gender','native-country']

# Feature Scaling
continuous_columns = ['age','fnlwgt', 'education-num','capital-gain','capital-loss','hours-per-week']

# 레이블 인코딩
target = 'income'

In [23]:
# import 
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [24]:
class_name = np.array(["5만달러 이하", "5만달러 초과"])

In [27]:
df = pd.read_csv('data/adult.data', 
                   header=None,
                   names=cols,
                   skipinitialspace=True,
                   na_values="?"
)
df.shape

(32561, 15)

In [28]:
# 결측치 제거
df.dropna(inplace=True)
df.shape

(30162, 15)

In [30]:
# 행삭제 후 index name을 다시 설정 (0부터 1씩 증가)
df.reset_index(drop=True, inplace = True) ## index 컬럼을 만들지 말고 빼서 행 인덱스로 냅두기
# 원래 index를 제거하고 자동증가 정수 index로 변경

## 현재 데이터셋은 reset_index가 필요없으나
## 고유 ID 값을 가지는 데이터셋의 경우 index를 재조정할 필요가 있을 수 있다.

In [31]:
# X,y를 분리
## y : 레이블 인코딩
y = LabelEncoder().fit_transform(df[target])
y[:10]

array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1])

In [32]:
# X 중 범주형 타입 feature에 대해 One-Hot Encoding 처리
ohe = OneHotEncoder(sparse_output=False) # handle_unknown="ignore" : 모르는 데이터에 대해 0으로 자동 처리
                                        # default는 "error" : 모르는 데이터에 대해 에러 발생
ohe_tmp = ohe.fit_transform(df[category_columns])
ohe_tmp.shape

(30162, 98)

In [33]:
# 원핫인코딩 결과 + 연속형 컬럼값 = Dataset
X = np.concatenate([ohe_tmp,df[continuous_columns].values], axis=1)
X.shape

(30162, 104)

In [34]:
## 위의 과정에서 
### (전처리 완료한) 범주형 데이터 + (그대로인) 연속형 데이터 => 1차 DataSet 완성이 됨.

### 이후 train, test 셋으로 분리한 이후 
### 연속형 데이터 전처리를 하고 최종 모델링을 한다.

In [35]:
# train, test set 분리
X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                   stratify=y,
                                                   random_state=0)
X_train.shape, X_test.shape

((22621, 104), (7541, 104))

In [36]:
### Standard Scaling
####### Train set 학습, Train/Test 변환
s_scale = StandardScaler()
X_train_scaled = s_scale.fit_transform(X_train)
X_test_scaled = s_scale.transform(X_test)

In [37]:
# 모델링 - Cross Validation
lr = LogisticRegression(max_iter=1000,random_state=0)
result_lr=cross_val_score(lr,
                         X_train_scaled, # feature(X, input data)
                         y_train, # target(y, output data, label)
                         scoring = "accuracy", # 평가지표
                         cv=5, # fold 개수
                         n_jobs=-1 # 병렬 처리시 사용할 cpu 프로세서 개수, -1: 전부다 사용
)

In [38]:
print(result_lr.mean())

0.8490779977626997


In [40]:
# 모델링 - SVC
svm = SVC(random_state=0)
result_svm=cross_val_score(svm,
                         X_train_scaled, # feature(X, input data)
                         y_train, # target(y, output data, label)
                         scoring = "accuracy", # 평가지표
                         cv=5, # fold 개수
                         n_jobs=-1 # 병렬 처리시 사용할 cpu 프로세서 개수, -1: 전부다 사용
)

In [41]:
print(result_svm.mean())

0.8448341320202628


In [42]:
## LogisticRegression 성능이 더 좋음 => 재학습
best_model = LogisticRegression(max_iter=1000, random_state=0)
best_model.fit(X_train_scaled, y_train)

LogisticRegression(max_iter=1000, random_state=0)

In [43]:
## 최종 평가

pred_test = best_model.predict(X_test_scaled)
accuracy_score(y_test, pred_test)

0.8456438138177961

In [44]:
class_name[pred_test]

array(['5만달러 초과', '5만달러 이하', '5만달러 이하', ..., '5만달러 이하', '5만달러 초과',
       '5만달러 이하'], dtype='<U7')

# 모델 저장
### `pickle` 모듈 활용!

> ## pickle 모듈사용
> - 객체 파일 입출력을 위한 파이썬 모듈
> - open() 시 **binary mode**로 설정한다.
> - 저장시 파일 확장자는 보통 `pkl` 이나 `pickle` 로 한다.
> - ex)
> ```python
> fw = open("data.pkl", "wb") # 객체를 pickle에 저장하기 위한 output stream 생성
> fr = open("data.pkl", "rb") # 파일에 저장된 객체를 읽어오기 위한 input stream 생성
> ```
> - **메소드**
>     - dump(저장할 객체, fw) : 출력
>     - load(fr): 입력 - 읽은 객체를 반환한다.

In [46]:
import os

save_root = 'adult_data_model'

ohe_path = os.path.join(save_root, 'ohe.pkl')
scaler_path = os.path.join(save_root, 'scaler.pkl')
model_path = os.path.join(save_root, 'model.pkl')

os.makedirs(save_root, exist_ok=True) # save_root 디렉토리를 생성


In [47]:
import pickle

In [48]:
###### 저장
# OheHotEcoder저장
with open(ohe_path, "wb") as fw1:
    pickle.dump(ohe, fw1) # (저장할 객체-값, 출력stream-wb)

# StandardScaler 저장
with open(scaler_path, "wb") as fw2:
    pickle.dump(s_scale, fw2)
    
# 모델 저장
with open(model_path, "wb") as fw3:
    pickle.dump(best_model, fw3)

In [49]:
###### 로딩
import pickle
# 모델 로딩
with open(model_path, 'rb') as fr1:
    save_model = pickle.load(fr1) # 읽어드릴 피클파일과 연결된 input stream
    
# OheHotEcoder loading
with open(ohe_path, 'rb') as fr2:
    save_ohe = pickle.load(fr2)
    
# scaler loading
with open(scaler_path, 'rb') as fr3:
    save_scaler = pickle.load(fr3)

In [51]:
from sklearn.metrics import accuracy_score
pred_test10 = save_model.predict(X_test_scaled)
accuracy_score(y_test, pred_test10)

0.8456438138177961

In [52]:
# 새로운 데이터 추론
new_data = df.iloc[:5]
new_data

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [53]:
# 전처리
### OneHotEncoding 이후 Standard Scaling

In [54]:
# Encoding
tmp = save_ohe.transform(new_data[category_columns])
tmp.shape

(5, 98)

In [55]:
# ohe + 연속형
tmp2 = np.concatenate([tmp,new_data[continuous_columns].values], axis=1)
tmp2.shape

(5, 104)

In [56]:
# Scaling
new_X = save_scaler.transform(tmp2)
new_X.shape

(5, 104)

In [57]:
new_pred = save_model.predict(new_X)
new_pred

array([0, 0, 0, 0, 1])

In [58]:
class_name[new_pred]

array(['5만달러 이하', '5만달러 이하', '5만달러 이하', '5만달러 이하', '5만달러 초과'], dtype='<U7')